# Setting Up Environment
```bash
$ python -m venv tempo
$ source tempo/bin/activate # for linux
$ ./tempo/Scripts/activate # for windows
$ python.exe -m pip install --upgrade pip # upgrade pip
$ pip install -r requirements.txt
```